Here, you develop the 

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [2]:
nh = 16
refRatio = 2
c = 1.
CFL = 0.5
nt = 32
RK = 4
deriv = 'CD'
order = 2

This cell contains the relevant switch information.

In [3]:
if (RK == 1):
    TimeIntegratorFunc = ST.ForwardEuler
else:
    if (RK == 2):
        TimeIntegratorFunc = ST.MidpointMeth
    else:
        TimeIntegratorFunc = ST.RK4

if (deriv == 'U'):
    DiffFunc = ST.Upwind
else:
    DiffFunc = TT.CenterDiff#ST.CenterDiff

This cell instantiates the grid, derivative operators, and Fourier modes.

In [4]:
omega = BT.Grid(nh)
finehalf = list(np.arange(int(nh / refRatio)))# + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_min = omega.nh_min
nh_max = omega.nh_max
t, nt = ST.CalcTime(omega, CFL, c, nt = nt)
derivMat = OT.SpaceDeriv(omega, order, deriv)
spatOp = -c * derivMat
strings = omega.strings
timePropOp = LA2.expm(t * spatOp)
omegaF = BT.Grid(nh_max)
wavesF = WT.MakeWaves(omegaF)

YOU'RE USING THE UPDATE.
bounds: [-1.   0.   0.5  1. ]

YOU'RE USING THE UPDATE.
bounds: [-1.  -0.5 -0.   1. ]



In [5]:
polyInterp = GTT.GhostCellStencil(2, -0.5)
print(polyInterp)
print(type(polyInterp))

YOU'RE USING THE UPDATE.
bounds: [-1.   0.   0.5  1. ]

(array([-0.5 ,  0.75, -0.25]), 1, 2)
<class 'tuple'>


This cell runs a test on a fifth-order cell-averaged polynomial interpolation for a ghost cell at the edge of the coarse-fine interface. If it doesn't reach the appropriate order of accuracy, there is an `assert` which force quits.

In [6]:
TT.TestPoly(5, -0.5)

YOU'RE USING THE UPDATE.
bounds: [-2.  -1.   0.   0.5  1.   1.5  2. ]
YOU'RE USING THE UPDATE.
bounds: [-2.  -1.   0.   0.5  1.   1.5  2. ]

p(x) =
  
2
P(x) =
  
2 x
Order 0:


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 6)

In [13]:
order = 3
for i in range(10):
    x_0 = 0.5 * (i + 1)
    print(x_0)
    print(np.polynomial.polynomial.polyvander(x_0, order)[0][::-1])
    print('')

0.5
[0.125 0.25  0.5   1.   ]

1.0
[2. 2. 2. 2.]

1.5
[10.125  6.75   4.5    3.   ]

2.0
[32. 16.  8.  4.]

2.5
[78.125 31.25  12.5    5.   ]

3.0
[162.  54.  18.   6.]

3.5
[300.125  85.75   24.5     7.   ]

4.0
[512. 128.  32.   8.]

4.5
[820.125 182.25   40.5     9.   ]

5.0
[1250.  250.   50.   10.]

